# 短歌を機械的に57577に分け、区切れを発見するのは可能か
1. 入力された短歌（57577の区切りの部分で分けられていない）に対し、mecabを用いた構文解析を行う
2. 得られた構文の情報をもとに再度57577に分けるプログラムを作成し、元の短歌との一致率を記録
3. 入力された短歌（57577の区切りの部分で予め分けられている）に対し、１と同様に構文解析を行い、句切れ（和歌用語の句切れ）をプログラムによって判断する

In [1]:
!pip install mecab-python3
!pip install unidic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 577 kB 7.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7425 sha256=97b26c0161166c86d25c885250ae99ce6ec6877a2bcf1b814807abf27f91bbf3
  Stored in directory: /root/.cache/pip/wheels/22/bc/bb/46aba36d0388f67dfe44bb0edc20a2c964560d4d19ec394e05
Successfully built unidic


In [2]:
!pycodestyle Natural_Language

/bin/bash: pycodestyle: command not found


In [3]:
from collections import defaultdict

In [4]:
# GoogleDrive上のファイルを扱うことができるようにする．
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
!python -m unidic download

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:21<00:00, 24.9MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.8/dist-packages/unidic/dicdir


In [5]:
# 使用するパスのまとめ
result_dir = "/content/drive/MyDrive/natural"# 結果の保存
hyaku_kugire_answer = "/content/drive/MyDrive/natural/hyaku_kugire.csv"
data_link = {}
data_link["hyaku"] = "/content/drive/MyDrive/natural/hyaku.csv"
data_link["kokin"] = "/content/drive/MyDrive/natural/kokin_data.csv"
data_link["shinkokin"] = "/content/drive/MyDrive/natural/shinkokin_data.csv"
data={}
# データの読み込み
for k in data_link.keys():
  with open(data_link[k],encoding="utf_8_sig") as f:
   tmp = f.read().split("\n")
  # 全角スペースを半角に
  for i in range(len(tmp)):
    tmp[i] = tmp[i].replace('\u3000', ' ').replace('－', ' ')
  
  data[k] = tmp

#print(data["kokin"])

In [6]:
# 2.で使用する和歌を５つの区切りに分けるためのクラス　一首一首の和歌のデータとして３．以降も使用
from time import perf_counter_ns
# main 関数 class版 確認用
import MeCab 
import unidic
class find5():
  def __init__(self,txt):
    self.txt = txt.replace('　', '').strip(' ').replace('\u3000', '')
    self.ideal = txt.replace(' ', '　').strip(' ').replace('\u3000', '　')# mecab用に空白を全角に
    self.tagger = MeCab.Tagger('-d /content/drive/MyDrive/natural/20_chuko/ ')
    self.mecab_result = (self.tagger.parse(self.txt)).split('\n')# mecabによる解析で得られたデータをそのまま格納
    for i in range(len(self.mecab_result)):
      self.mecab_result[i] = self.mecab_result[i].split(',')
    self.result_2 = (self.tagger.parse(self.ideal)).split('\n') # 区分けずみのデータをmecabで解析したもの
    for i in range(len(self.result_2)):
      self.result_2[i] = self.result_2[i].split(',')
    self.joint = [5,7,5,7,7] # 基準点あとで
    self.not_start = ["助詞","助動詞","み	接尾辞"]# 句が始まらない語
    self.yomi = []# カタカナの読み仮名
    self.complete = ""# いる？　再度区分けされたやつ
    self.divided = []# 何語目で区切れるか
    self.answer_divided = []# 区切りの正解
    self.word = []# 一茶くん的なあとで
   
  def getAnswer(self): # 正解の区切りのインデックスを登録
    self.answer_divided = [-1]
    count = 0 
    for i in range(len(self.result_2)):
      if ("空白" in self.result_2[i][0]):# 区切れを発見
        self.answer_divided.append(count-1)
      else:
        count += 1
    self.answer_divided.append(len(self.result_2)-7)# 最後の部分の調整3+空白の４ 
    return self.answer_divided  
  def change_to_yomi(self):
    if(self.yomi != []):
      return
    self.yomi = []
    for i in range(len(self.mecab_result)):
      
      if(len(self.mecab_result[i]) >= 9):
        self.yomi.append(self.mecab_result[i][9])
    return self.yomi
  def judge_joint(self, i):# 次の句がnot_startで始まらないようにする、ある単語がnot_startの時１を返し、自立語なら０
    for s in self.not_start:
      if(s in self.mecab_result[i][0]):
        if(s == "み	接尾辞"):
          if(i-1 == 0 or "語幹"not in self.mecab_result[i-1][5]):
            return 0# 接尾辞の「み」ではないと判断
        return 1
    return 0
  def loose_div(self, i):
    # 句切れ分けの制約がすくないバージョン
    letter = 0
    while(letter < self.joint[len(self.divided)-1]):
    
      letter += len(self.yomi[i])
      if(letter == self.joint[len(self.divided)-1]):# 57577の区切れと単語の区切れが一致した際はそこで分ける
        return i
      elif(letter > self.joint[len(self.divided)-1]):# 7577の区切れと単語の区切れが一致せず、それより字数が多くなってしまった場合は字数が近い方で分ける
        if((letter - self.joint[len(self.divided)-1]>=
            self.joint[len(self.divided)-1] - (letter-len(self.yomi[i])))):
          return i-1
        else:
          return i  
      i+=1
  def divide(self, i):
    letter = 0 # 何文字目かを示す
    while(letter < self.joint[len(self.divided)-1]):
      
      letter += len(self.yomi[i])
      if(letter == self.joint[len(self.divided)-1]):
        # 57577の区切れと単語の区切れが一致した際も次の単語がnoneselfである場合は今の句に含む
        flag = find5.judge_joint(self, i+1)
        if(flag == 0):
          return i
        else:
          while(flag == 1):
            i += 1
            flag = find5.judge_joint(self, i+1)# not_startに含まれない語の直前まで進む
        return i
      elif(letter>self.joint[len(self.divided)-1]):# 57577の区切れと単語の区切れが一致せず、それより字数が多くなってしまった場合
        if(find5.judge_joint(self, i) == 1):# 今注目している要素はnot_start、not_startに含まれない語の直前まで進む
          flag = find5.judge_joint(self, i+1)
          while(flag == 1):
            i += 1
            flag=find5.judge_joint(self, i+1)
          return i
        else: # not_start＋not_startまたはそれ以外＋それ以外
          if("名詞" in self.mecab_result[i-1][0] 
             and "名詞" in self.mecab_result[i][0]):# 両方とも名詞のときはもとは一つの名詞であると判断
            return i
          elif(find5.judge_joint(self, i+1) == 1 or
               (letter-self.joint[len(self.divided)-1] >=
                self.joint[len(self.divided)-1] - (letter-len(self.yomi[i])))):# 次がnot_start内の単語なら前の方を取る。次もそれ以外なら音数が近い方で判断
            return i-1
          else:
            return i 
      i += 1
  def div5(self):
    self.change_to_yomi()
    self.divided = [-1]# 57577の手前の単語の番号
    i = 0
    while(i < len(self.mecab_result) and len(self.divided) < 5):
      i = find5.divide(self,i)
      self.divided.append(i)
      i += 1
    self.divided.append(len(self.yomi) - 1)
    self.divided = sorted(set(self.divided))
    self.divided = list(self.divided)# 重複要素の削除
    self.warning()# あとで
    return self.divided
  def output_complete(self):
    self.complete = ""
    for i in range(len(self.mecab_result)-2):
      self.complete += self.mecab_result[i][8]
      if(i in self.divided):
        self.complete += ' '  
    self.complete = self.complete.replace(' ','　').replace('\u3000','　')
    self.complete = self.complete.replace('\u3000','　').strip(' ').strip('　')
    return self.complete# 入力を57577に分けた文字列を出力
  def longest_div(self):
    #最長の句を特定
    tmp = 0
    max = 0
    index=-1
    if(len(self.divided) == 0):
      print("not divided: ",self.change_to_yomi())
      return 0
    for i in range(0, len(self.divided) - 1):
      tmp = 0
      for j in range(self.divided[i] + 1, self.divided[i + 1] + 1):
        tmp += len(self.yomi[j])
      if(tmp > max):
        max = tmp
        index = i
    print(index+1,"句目が最長")
    print(self.yomi[self.divided[i] + 1][0])
    return index
  def modify_div(self):
      i=find5.longest_div(self)
      letter=0
      # 制約をゆるめて区分けする
      j=find5.loose_div(self,self.divided[i]+1)
      self.divided.append(j)
      self.divided=sorted(set(self.divided))
      if(len(self.divided)<6):
        self.divided.append(-2)# 大規模な句またがりが生じていると考えられる　別個に対処
      return self.divided
  def warning(self):
    prev = 7
    current = len(self.divided)
    #終了後5句に分けられていない場合、最も長い「句」を二つにくぎる
    while(current < 6 and -2 not in self.divided):
      #二か所以上で区分できていない可能性を考慮
      #もっとも長い句を二つに分けることを繰り返す
      prev = len(self.divided)
      self.divided = find5.modify_div(self)
      current = len(self.divided)
    self.complete = find5.output_complete(self)
    if(-2 in self.divided):
      print("句またがりが", str(find5.longest_div()+1),
            "句目で起きていると考えられます")
      if(len(self.complete) < 25):
        print("俳句を入力していませんか？")
      print(self.change_to_yomi())
  def divide_and_compare(self):
    #（生成される文字列があっているか,何文節目かの区切りがあっているか）
    self.getAnswer()
    self.div5()
    #print(self.divided)
    #print(self.answer_divided)
    #print(self.complete)
    #print(self.ideal)
    #print(len(self.complete))
    #print(len(self.ideal))
    #print(self.yomi)
    if(self.complete == self.ideal):
      if(self.divided == self.answer_divided):
        return 1,1
      return 1,0 #一致
    else:
      if(self.divided[1:5] == self.answer_divided[1:5]):
        return 0,1
      return 0,0

In [24]:
class find1(find5):
  def __init__(self,result):  # コンストラクタと呼ばれる
        super().__init__(result);
        self.point = 0
        self.stop = ["終止", "命令", "已然"]
        self.kakari = ["ぞ", "なむ", "や", "か", "こそ"]	
        self.edge = ["な	助詞", "か	助詞", "かな	助詞", "かも	助詞", "かし	助詞", "ぞ	助詞", "よ	助詞", "ものを	助詞", "もや	助詞", "もよ	助詞", "やし	助詞", "ろ	助詞", "ゑ	助詞"]
        self.candidate = []# 句切れになりそうなもの
  def find_end(self):# ①
    for i in range(1, 5):
      for s in self.stop:
        if(s in  self.mecab_result[self.answer_divided[i]][5]):
          self.candidate.append(i)
     
      
  def find_kakari(self):# ②
    flag_r = 0# 連体形
    flag_e = 0# 已然形
    for i in range(len(self.mecab_result) - 2):
   
      for s in self.kakari:
        
        if(s in self.mecab_result[i][0]):
         
          if(s == "こそ"):
            
            flag_e = 1
          else:
           
            flag_r = 1
      if(flag_r == 1):
        if("連体" in self.mecab_result[i][5]):
          
          flag_r = 0
          if(i in self.answer_divided):
            self.candidate.append(self.answer_divided.index(i))
      if(flag_e==1):
       
        if("已然" in self.mecab_result[i][5]):
          flag_e=0
          if(i in self.answer_divided):
            self.candidate.append(self.answer_divided.index(i))

      
  def find_edge(self):
    for i in range(1, 5):
     
      for s in self.edge:
        if(s in  self.mecab_result[self.answer_divided[i]][0]):
          self.candidate.append(i)
  def find_standard(self):
    
    find1.find_kakari(self)
    
    find1.find_edge(self)
   

    find1.find_end(self)
    self.candidate = sorted(set(self.candidate))
    self.candidate = list(self.candidate)# 重複要素の削除
    if(self.candidate == []):
      self.point = 5
    else:
      self.point = self.candidate[0]
    
      






  
  
  


In [7]:
sample_txt = "みちしらは たつねもゆかむ もみちはを ぬさとたむけて あきはいにけり"
sample_data = find5(sample_txt)
print(sample_data.ideal)
print(sample_data.mecab_result)

print(sample_data.divide_and_compare())



みちしらは　たつねもゆかむ　もみちはを　ぬさとたむけて　あきはいにけり
[['みち\t名詞', '固有名詞', '地名', '一般', '*', '*', 'ミチ', 'ミチ', 'みち', 'ミチ', 'ミチ', '固', 'みち', 'ミチ', 'ミチ', 'ミチ', '*', '*', '*', '*', '*', '*', '1', '*', '*', '54778502586704384', '199283'], ['しら\t副詞', '*', '*', '*', '*', '*', 'シラ', 'しら', 'しら', 'シラ', 'シラ', '和', 'しら', 'シラ', 'シラ', 'シラ', '*', '*', '*', '*', '*', '*', '2', '*', '*', '74398746384933376', '270661'], ['は\t助詞', '係助詞', '*', '*', '*', '*', 'ハ', 'は', 'は', 'ワ', 'ハ', '和', 'は', 'ワ', 'ハ', 'ハ', '*', '*', '*', '*', '*', '*', '*', '"動詞%F2@0', '名詞%F1', '形容詞%F2@-1"', '*', '8059703733133824', '29321'], ['たつ\t動詞', '一般', '*', '*', '文語四段-タ行', '終止形-一般', 'タツ', '立つ', 'たつ', 'タツ', 'タツ', '和', 'たつ', 'タツ', 'タツ', 'タツ', 'タ濁', '基本形', '*', '*', '*', '*', '1', 'C1', '*', '6167186557248171', '22436'], ['ね\t助動詞', '*', '*', '*', '文語助動詞-ズ', '已然形-一般', 'ズ', 'ず', 'ね', 'ネ', 'ネ', '和', 'ず', 'ズ', 'ズ', 'ズ', '*', '*', '*', '*', '*', '*', '*', '"形容詞%F4@-1', '動詞%F3@0"', '*', '5384042186941185', '19587'], ['も\t助詞', '係助詞', '*', '*', '*', '*', 'モ', '

In [14]:
# 機械的な区分けの精度を確認
mechanical_accuracy = {}
song_data = {}
for k in data.keys():
  song_data[k] = []
  print(k)
  count = 0
  letter = 0
  number = 0
  #print(data[k])
  for i in range(len(data[k])-1):
    song = data[k][i]
    tmp = find1(song)
    a,b = tmp.divide_and_compare()
    letter += a
    number += b
    count += 1
    song_data[k].append(tmp)# あとで使う
  #print(letter)
  #print(number)
  #print(count)
  letter = float(letter)/count
  number = float(number)/count
  print("文字数で判断した場合の一致率は",letter)
  print("文節で判断した場合の一致率は",number)
  mechanical_accuracy[k] = (letter,number)





hyaku
2 句目が最長
イ
文字数で判断した場合の一致率は 0.97
文節で判断した場合の一致率は 0.93
kokin
1 句目が最長
オ
3 句目が最長
ウ
4 句目が最長
チ
2 句目が最長
ヒ
4 句目が最長
オ
4 句目が最長
イ
2 句目が最長
オ
3 句目が最長
ク
2 句目が最長
カ
文字数で判断した場合の一致率は 0.7461746174617462
文節で判断した場合の一致率は 0.6174617461746175
shinkokin
3 句目が最長
ヨ
2 句目が最長
ツ
1 句目が最長
マ
4 句目が最長
ム
4 句目が最長
ヨ
4 句目が最長
オ
2 句目が最長
イ
4 句目が最長
コ
2 句目が最長
カ
4 句目が最長
マ
1 句目が最長
ム
4 句目が最長
ツ
4 句目が最長
カ
4 句目が最長
ト
4 句目が最長
シ
文字数で判断した場合の一致率は 0.7401496259351621
文節で判断した場合の一致率は 0.6034912718204489


In [21]:
print(mechanical_accuracy)
print(song_data["kokin"][0].answer_divided)

{'hyaku': (0.97, 0.93), 'kokin': (0.7461746174617462, 0.6174617461746175), 'shinkokin': (0.7401496259351621, 0.6034912718204489)}
[-1, 3, 8, 11, 16, 21]


In [30]:
point_count = {}
for k in song_data.keys():
  point_count[k] = [0] * 6
  #print(data[k].pure_revised)
  for i in range(len(song_data[k])-1):
    song_data[k][i].find_standard()
    point_count[k][song_data[k][i].point-1] += 1
  
  '''
  if(data[k].point==h_k[k]):
    pre+=1
  else:
    print(data[k].point)
    print(h_k[k])
    print(ideal)
  '''

  '''
  with open(result_dir + k + ".csv", "w", encoding="utf_8_sig") as f:
    for i in range(len(song_data[k])-1):
      f.write(f"{i},{song_data[k][i].point}\n")
  files.download(result_dir + k + ".csv")
  '''
    

In [41]:
for k in song_data.keys():
  print(k)
  sum = 0
  for i in range(5):
    sum += point_count[k][i]
  for i in range(5):
    point_count[k][i] /= float(sum)
  
  print(point_count[k])
 
 

hyaku
[0.06060606060606061, 0.15151515151515152, 0.10101010101010101, 0.08080808080808081, 0.6060606060606061, 0]
kokin
[0.06036036036036036, 0.18198198198198198, 0.12072072072072072, 0.0963963963963964, 0.5405405405405406, 0]
shinkokin
[0.10778443113772455, 0.13223552894211577, 0.1472055888223553, 0.11676646706586827, 0.49600798403193613, 0]


In [ ]:
human_point = {}
human_point["kokin"] = [0.013636, 0.175455, 0.16, 0.061818, 0.589091] # 人力との相関は0.99
human_point["shinkokin"] = [0.047017, 0.092518, 0.218908, 0.066229, 0.575329] # 人力との相関は0.97